## Basic Background Subtraction using [OpenCV](https://www.opencv.org/)

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://www.opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use [CMake](https://www.cmake.org/).

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
// #include <vector>
#include <stdexcept>
// #include <sstream>
// #include <string>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Open the webcam

We are going to use the [`cv::VideoCapture` class](https://docs.opencv.org/master/d8/dfe/classcv_1_1VideoCapture.html) to grab frames from the webcam.

In [5]:
// Open the default camera (see the 0 below)
VideoCapture video_input(0);

// Check VideoCapture documentation.
if (!video_input.isOpened())
    throw runtime_error("OpenCV found no webcam, the program will terminate");

## Display the images from the camera

We create a window to display the images from the webcam

In [6]:
namedWindow("Webcam", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Background", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Absolute difference", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Foreground mask", WINDOW_GUI_EXPANDED); // Create a window
namedWindow("Foreground", WINDOW_GUI_EXPANDED); // Create a window

We display the images in a loop. We also need the background. We pick up the backbround by pressing the 'b' key on the keyboard.

In [7]:
Mat background;

We'll perform the background subtraction using an absolute difference between the background and the incoming frame. Then, we apply a threshold. It creates a binary mask of the foreground. We write the background subtraction as a function. But first, as we'll create a binary mask, I want to do some cleaning using mathematical morphology and use it via a new function.

In [8]:
Mat cleanBinaryImage(const Mat& aBinaryImage, int elementSize = 5)
{
    Mat output;
    
    Mat element = getStructuringElement(MORPH_ELLIPSE,
                        Size(elementSize, elementSize));
    
    morphologyEx(aBinaryImage, output, MORPH_CLOSE, element);
    morphologyEx(output, output, MORPH_OPEN, element);

    return output;
}

In [9]:
Mat getForegroundMask(const Mat& aBackground, const Mat& aNewFrame, int aThreshold = 128)
{
    // Convert in greyscale
    Mat grey_frame;
    cv::cvtColor(aNewFrame, grey_frame, COLOR_BGR2GRAY);
    
    // Blur the image a bit
    medianBlur(grey_frame, grey_frame, 5);

    // Convert to float32
    grey_frame.convertTo(grey_frame, CV_32F);

    // Compute the foreground as the absolute difference
    Mat foreground;
    foreground = aBackground - grey_frame;
    foreground = abs(foreground);
    
    // Normalise the foreground
    normalize(foreground, foreground, 0, 255, NORM_MINMAX, CV_8UC1);          

    // Display the foreground
    imshow("Absolute difference", foreground);

    // Apply a threshold to generate the foreground mask
    Mat foreground_mask;
    threshold( foreground, foreground_mask, aThreshold, 255,  THRESH_BINARY);

    // Use mathematical morphology to clean the binary image
    foreground_mask = cleanBinaryImage(foreground_mask, 15);

    // Display the foreground mask
    imshow("Foreground mask", foreground_mask);

    // Return the mask
    return foreground_mask;
}

As we'll use a threshold, I'll add a trackbar to control it.

In [10]:
int threshold_value = 128;
createTrackbar( "Threshold: ", "Foreground mask", &threshold_value, 255, NULL );

In [11]:
int key = -1;
while (key != 27 && key != 'q')
{
    // Grab a new frame
    Mat frame;
    video_input >> frame;
    
    // Make sure everything went well
    if (frame.empty())
    {
        video_input.release(); // We are now done with the camera, stop it
        throw runtime_error("OpenCV cannot grab a new frame from the camera, the program will terminate");
    }
    // There is frame
    else
    {
        // Display the image
        imshow("Webcam", frame);

        // This is the background
        if (key == 'b')
        {
            background = frame;
            cv::cvtColor(background, background, COLOR_BGR2GRAY);
            imshow("Background", background);
            medianBlur(background, background, 5);
            background.convertTo(background, CV_32F);
        }
        // This is not the background
        else
        {
            // The background exists
            if (!background.empty())
            {
                Mat foreground_mask = getForegroundMask(background, frame, threshold_value);

                // Apply the foreground mask
                Mat clean;
                frame.copyTo(clean, foreground_mask);
                imshow("Foreground", clean);
            }
        }    
    }
    
    key = waitKey(1);
}

We don't need the window, destroy it

In [12]:
destroyAllWindows(); // Destroy all the created windows

We are now done with the camera, stop it

In [13]:
video_input.release();

## Screenshot

![Screenshot](../screenshot.png)